In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Створення клієнта

У цьому прикладі ми будемо використовувати [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) для доступу до LLM.

`model` визначено як `gpt-4o-mini`. Спробуйте змінити модель на іншу, доступну на маркетплейсі GitHub Models, щоб побачити різні результати.

Для швидкого тесту ми просто запустимо простий запит - `What is the capital of France`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Визначення функцій

У цьому прикладі ми надамо агенту доступ до інструмента, який є функцією зі списком доступних туристичних напрямків та їхньою доступністю.

Можна уявити, що це буде сценарій, у якому туристичний агент може мати доступ до туристичної бази даних, наприклад.

Під час ознайомлення з цим прикладом не соромтеся спробувати визначити нові функції та інструменти, які агент може викликати.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Визначення інструменту функції
Щоб агент міг використовувати `vacation_destinations` як `FunctionTool`, нам потрібно визначити його як такий.

Ми також надамо опис цього інструменту, що допоможе агенту зрозуміти, для чого цей інструмент використовується у зв’язку із завданням, яке запросив користувач.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Визначення агента

Тепер ми можемо створити агента в наведеному нижче коді. Ми визначаємо `system_message`, щоб надати агенту інструкції щодо допомоги користувачам у пошуку місць для відпочинку.

Ми також встановлюємо параметр `reflect_on_tool_use` на значення true. Це дозволяє використовувати LLM для отримання відповіді від виклику інструменту та надсилання відповіді природною мовою.

Ви можете встановити цей параметр на значення false, щоб побачити різницю.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Запуск агента

Тепер ми можемо запустити агента з початковим повідомленням користувача, в якому запитується поїздка до Токіо.

Ви можете змінити це місто, щоб побачити, як агент реагує на доступність міста.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
